In [ ]:
# TODO documentation
# Based on https://github.com/seanrjohnson/protein_scoring/blob/main/colab_notebooks/Protein_Metrics.ipynb

In [ ]:
import os

!pip -q install  git+https://github.com/facebookresearch/esm.git@e5e7b06b9a093706607c229ab1c5c9821806814d

if (not os.path.isdir("protein_gibbs_sampler")):
    !git clone https://github.com/seanrjohnson/protein_gibbs_sampler > /dev/null
    !cd protein_gibbs_sampler ; git checkout 7de11ff63f7dca3cb52ef84109152ac8f988b7a0 2>&1 > /dev/null ; yes | pip install -q --no-dependencies . > /dev/null

!yes | pip -q install  biopython~=1.79 pytest-datadir~=1.4.1 > /dev/null

In [ ]:
# Read the input file
from Bio import SeqIO

input_file = 'basemodel-RITA_l-generated.fasta'
dataset = list(SeqIO.parse(input_file, "fasta"))

In [ ]:
# Performing the "Quality Checks"
# TODO change order of the filters to do the computationally expensive last
from phobius import scrape_phobius_for_transmembrane_domains
import numpy as np

# remove seqences that don't start with methionine
dataset = [seq for seq in dataset if str(seq.seq).startswith('M')]

# remove sequences with long repeats
def find_longest_repeat(seq, k):
  longest = [1] * len(seq)
  pattern = [None] * len(seq)
  
  seq_len = len(seq)
  for i in range(seq_len):
    if i + k <= seq_len:
      pattern[i] = seq[i:i+k]
    if i - k >= 0:
      if pattern[i-k] == pattern[i]:
        longest[i] = longest[i-k] + 1
  return max(longest)

no_long_repeats = []
for sequence in dataset:
  if (find_longest_repeat(str(sequence.seq), 1) <= 3 and find_longest_repeat(str(sequence.seq), 1) <= 4):
    no_long_repeats.append(sequence)

dataset = no_long_repeats

# remove sequences with predicted transmembrane domains
# hint - scraping might take several seconds to a few minutes, depending on the dataset size
transmembrane_mask = scrape_phobius_for_transmembrane_domains(dataset)
dataset = np.array(dataset)
# transmembrane_mask is an array of boolean predictions, so we use its negation it to only retain sequences without predicted transmembrane domains
dataset = np.array(dataset)[~transmembrane_mask]

In [ ]:
from protein_gibbs_sampler.src.pgen.likelihood_esm import main as likelihood_esm
from io import StringIO

output_file = 'out/esm_results.csv'

input_handle = StringIO()
SeqIO.write(dataset, input_handle, 'fasta')
output_handle = open(output_file, 'w')

# TODO fails if a sequences contains more than 1024 amino acids or ambiguous AA characters
# TODO can't read lines from StringIO
likelihood_esm(input_handle, output_handle, True, 'cpu', 'esm1v', 1, float('inf'), True, 'score', None)
